In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import math

In [7]:
df = pd.read_csv("dataset_output/libras_minds/libras_minds_openpose_80_frames.csv")

In [8]:
df = df.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1)

In [4]:
person_videos = 15
persons = 5
frames = 30
person = []
for i in range(persons):
    person.extend([i] * frames * person_videos)
person = person * len(df["category"].unique())

In [20]:
person = df["video_name"].str[-8:-6]

In [21]:
df["person"] = person

In [22]:
df

,category,video_name,frame,hand_0_0_x,hand_0_0_y,hand_0_0_z,hand_0_1_x,hand_0_1_y,hand_0_1_z,hand_0_2_x,...,pose_22_z,pose_23_x,pose_23_y,pose_23_z,pose_24_x,pose_24_y,pose_24_z,missing_hand,missing_face,person
0,Acontecer,01AcontecerSinalizador01-1.mp4,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,01
1,Acontecer,01AcontecerSinalizador01-1.mp4,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,01
2,Acontecer,01AcontecerSinalizador01-1.mp4,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,False,01
3,Acontecer,01AcontecerSinalizador01-1.mp4,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,False,01
4,Acontecer,01AcontecerSinalizador01-1.mp4,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,False,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92635,Vontade,20VontadeSinalizador12-5.mp4,162,0.614689,0.910987,0.588928,0.598457,0.922928,0.627919,0.588381,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,12
92636,Vontade,20VontadeSinalizador12-5.mp4,163,0.614785,0.908597,0.584819,0.598921,0.922699,0.580145,0.588155,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,12
92637,Vontade,20VontadeSinalizador12-5.mp4,164,0.614690,0.911093,0.602180,0.598085,0.925853,0.639054,0.588122,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,12
92638,Vontade,20VontadeSinalizador12-5.mp4,165,0.614768,0.910723,0.575162,0.599248,0.924518,0.586799,0.588717,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,12


In [23]:
df["category"].unique()

array(['Acontecer', 'Aluno', 'Amarelo', 'America', 'Aproveitar', 'Bala',
       'Banco', 'Banheiro', 'Barulho', 'Cinco', 'Conhecer', 'Espelho',
       'Esquina', 'Filho', 'Maca', 'Medo', 'Ruim', 'Sapo', 'Vacina',
       'Vontade'], dtype=object)

In [24]:
landmarks_name = np.array([i for i in list(df.columns) if i not in ["Unnamed: 0", "Unnamed: 0.1", "Unnamed: 0.2", "category", "category_index", "video_name", "frame", "missing_hand", "missing_face", "person"]])
landmarks_name = landmarks_name.reshape((int(landmarks_name.shape[0]/3), 3))

In [25]:
# GPT Code
def rotate_landmarks(landmarks, rotation_angle):
    # Assuming landmarks are in (x, y, z) format
    
    # Calculate the centroid (center) of the landmarks
#     centroid = np.mean(landmarks, axis=0)
    centroid = 0.5

    # Translate landmarks to the origin (center)
    translated_landmarks = landmarks - centroid
    # Create a rotation matrix for the given angle (in radians)
    rotation_matrix = np.array([[np.cos(rotation_angle), -np.sin(rotation_angle), 0],
                                 [np.sin(rotation_angle), np.cos(rotation_angle), 0],
                                 [0, 0, 1]])
    
    # Apply the rotation to each landmark
#     print(rotation_matrix)
    rotated_landmarks = np.dot(translated_landmarks, rotation_matrix)
    
    return rotated_landmarks + centroid

In [26]:
def zoom_landmarks(landmarks, zoom_factor):
    # Scale the landmarks by the zoom factor
    zoomed_landmarks = landmarks * zoom_factor
    
    return zoomed_landmarks

In [27]:
def translate_landmarks(landmarks, translation_vector):
    # Translate the landmarks by the given vector
    translated_landmarks = landmarks + translation_vector
    
    return translated_landmarks

In [28]:
def apply_transformation(frame, rotation, zoom, translate_x, translate_y):
    for lx, ly, lz in landmarks_name:
        result = rotate_landmarks(frame[[lx, ly, lz]], math.radians(rotation))
        frame[[lx, ly, lz]] = result
        result = zoom_landmarks(frame[[lx, ly, lz]], zoom)
        frame[[lx, ly, lz]] = result
        result = translate_landmarks(frame[[lx, ly, lz]], [translate_x, translate_x, 0])
        frame[[lx, ly, lz]] = result
    return frame

In [29]:
np.random.normal(0, 0.5, 10)

array([-0.23799893, -0.83642228, -0.91409889,  0.28544186,  0.10699852,
       -0.04809587, -0.83051297,  0.12134307,  0.06505087, -0.01918122])

In [30]:
sample_size = 0.4
samples_creation = 4
rotation_sigma = 4
zoom_sigma = 0.05
translate_x_sigma = 0.01
translate_y_sigma = 0.0

In [31]:
df_augmented = pd.DataFrame(columns=df.columns)
for category in df["category"].unique():
    print("Processing category", category)
    df_category = df[df["category"] == category]
    videos = df_category["video_name"].unique()
    videos = np.random.choice(videos, int(len(videos) * sample_size))
    for i in range(samples_creation):
        for video in videos:
            df_video = df[df["video_name"] == video].copy()
            rotation = np.random.normal(0, rotation_sigma)
            zoom = np.random.normal(0, zoom_sigma) + 1
            translate_x = np.random.normal(0, translate_x_sigma)
            translate_y = np.random.normal(0, translate_y_sigma)
            df_video = apply_transformation(df_video, rotation, zoom, translate_x, translate_y)
#             for index, frame in df_video.iterrows():
#                 frame = apply_transformation(frame, rotation, zoom, translate_x, translate_y)
#                 df_video[index] = frame
            df_video["video_name"] = f"augmented{i}-" + df_video["video_name"]
            df_augmented = pd.concat([df_augmented, df_video])

Processing category Acontecer
Processing category Aluno
Processing category Amarelo
Processing category America
Processing category Aproveitar
Processing category Bala
Processing category Banco
Processing category Banheiro
Processing category Barulho
Processing category Cinco
Processing category Conhecer
Processing category Espelho
Processing category Esquina
Processing category Filho
Processing category Maca
Processing category Medo
Processing category Ruim
Processing category Sapo
Processing category Vacina
Processing category Vontade


In [32]:
df_augmented

,category,video_name,frame,hand_0_0_x,hand_0_0_y,hand_0_0_z,hand_0_1_x,hand_0_1_y,hand_0_1_z,hand_0_2_x,...,pose_22_z,pose_23_x,pose_23_y,pose_23_z,pose_24_x,pose_24_y,pose_24_z,missing_hand,missing_face,person
4240,Acontecer,augmented0-01AcontecerSinalizador11-4.mp4,1,0.635598,0.925130,0.692857,0.622417,0.945045,0.804851,0.615140,...,0.0,-0.008476,0.021696,0.0,-0.008476,0.021696,0.0,False,False,11
4241,Acontecer,augmented0-01AcontecerSinalizador11-4.mp4,3,0.635638,0.928058,0.697241,0.622948,0.946009,0.820685,0.615092,...,0.0,-0.008476,0.021696,0.0,-0.008476,0.021696,0.0,False,False,11
4242,Acontecer,augmented0-01AcontecerSinalizador11-4.mp4,5,0.635275,0.927866,0.734579,0.622329,0.944445,0.872368,0.615843,...,0.0,-0.008476,0.021696,0.0,-0.008476,0.021696,0.0,False,False,11
4243,Acontecer,augmented0-01AcontecerSinalizador11-4.mp4,7,0.635230,0.928952,0.719339,0.622155,0.946694,0.873436,0.614968,...,0.0,-0.008476,0.021696,0.0,-0.008476,0.021696,0.0,False,False,11
4244,Acontecer,augmented0-01AcontecerSinalizador11-4.mp4,9,0.635777,0.927947,0.698028,0.622187,0.946670,0.862803,0.614998,...,0.0,-0.008476,0.021696,0.0,-0.008476,0.021696,0.0,False,False,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89115,Vontade,augmented3-20VontadeSinalizador04-1.mp4,156,0.661128,0.869466,0.093740,0.648234,0.894273,0.206741,0.640855,...,0.0,-0.047766,0.008831,0.0,-0.047766,0.008831,0.0,False,False,04
89116,Vontade,augmented3-20VontadeSinalizador04-1.mp4,157,0.654486,0.885906,0.132987,0.640599,0.896898,0.282548,0.636257,...,0.0,-0.047766,0.008831,0.0,-0.047766,0.008831,0.0,False,False,04
89117,Vontade,augmented3-20VontadeSinalizador04-1.mp4,158,0.654151,0.895033,0.114622,0.638526,0.901083,0.239837,0.636232,...,0.0,-0.047766,0.008831,0.0,-0.047766,0.008831,0.0,False,False,04
89118,Vontade,augmented3-20VontadeSinalizador04-1.mp4,159,0.655912,0.890127,0.152973,0.643040,0.901179,0.265749,0.638438,...,0.0,-0.047766,0.008831,0.0,-0.047766,0.008831,0.0,False,False,04


In [22]:
pd.concat([df, df_augmented]).to_csv("dataset_output/libras_minds/libras_minds_openpose_80_frames_augmented.csv")

In [126]:
df_augmented.to_csv("datasets/lsa64_augmentation_test.csv")